In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd 

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.implicitly_wait(5)

# Define the product URL (replace with the actual product URL you want to scrape)
product_url = 'https://www.flipkart.com/oppo-f27-pro-dusk-pink-256-gb/p/itm5235c3c836cc9?pid=MOBHF3DJ8CREZW7M&lid=LSTMOBHF3DJ8CREZW7MGWIS9Y&marketplace=FLIPKART&q=phone+in+30000&store=tyy%2F4io&srno=s_1_5&otracker=AS_Query_HistoryAutoSuggest_1_1_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_1_na_na_na&fm=search-autosuggest&iid=c5368073-c127-4c37-837d-acd376505f45.MOBHF3DJ8CREZW7M.SEARCH&ppt=sp&ppn=sp&ssid=535nb6x24g0000001727865775042&qH=33eb63846b096281'

# Open the product page
driver.get(product_url)
driver.maximize_window()

try:
    product_name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span'))
    ).text
    product_id = product_name.strip()
    print(f"Product ID (Name): {product_id}")
except Exception as e:
    print(f"Error retrieving product name: {e}")
    driver.quit()

# Scroll down to make sure the "View All Reviews" button is visible
for _ in range(6):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

# Click on the "View All Reviews" button using XPath
try:
    view_all_reviews_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[7]/div/a/div/span'))
    )
    view_all_reviews_button.click()
    time.sleep(3)  # Wait for the reviews page to load
except Exception as e:
    print(f"Error finding 'View all reviews' button: {e}")
    driver.quit()

# Collect review data
reviews = []

try:
    # Loop through multiple review pages
    page_number = 1  # Initialize page counter
    
    while True:
        # Get the individual review containers (10 per page) using XPath
        review_containers = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="cPHDOP col-12-12"]/div[@class="EKFha-"]'))
        )

        # Scroll down to ensure all reviews on the page are loaded (optional if necessary)
        for _ in range(30):
            body_element = driver.find_element(By.TAG_NAME, 'body')
            body_element.send_keys(Keys.ARROW_DOWN)
            time.sleep(1)

        # Iterate through each review container and extract data using XPath
        for container in review_containers:
            try:
                review_text = container.find_element(By.XPATH, './/div[@class="ZmyHeo"]').text
            except:
                review_text = 'N/A'

            try:
                rating = container.find_element(By.XPATH, './/div[@class="XQDdHH Ga3i8K"]').text
            except:
                rating = 'N/A'

            reviews.append({
                'product_id': product_id,
                'review_text': review_text,
                'rating': rating,
            })

        # Dynamically change XPath for the "Next" button based on the page number
        try:
            if page_number == 1:
                next_button_xpath = '//*[@id="container"]/div/div[3]/div/div/div[2]/div[13]/div/div/nav/a[11]/span'
            else:
                next_button_xpath = '//*[@id="container"]/div/div[3]/div/div/div[2]/div[13]/div/div/nav/a[12]/span'
            
            # Wait for the "Next" button to become clickable and click it
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, next_button_xpath))
            )
            next_button.click()
            time.sleep(5)  # Add a delay to wait for the next page to load

            # Increment the page number after clicking the next button
            page_number += 1

        except Exception as e:
            print(f"No more pages found or an error occurred: {e}")
            break

except Exception as e:
    print(f"Error occurred: {e}")

# Print the collected reviews
for i, review in enumerate(reviews):
    print(f"Product ID: {review['product_id']}")
    print(f"Rating: {review['rating']}")
    print(f"Review Text: {review['review_text']}")
    print('-' * 50)

# Create a DataFrame from the reviews list and save it as a CSV
df = pd.DataFrame(reviews)

# Save the DataFrame to a CSV file
csv_filename = f'{product_id}_reviews.csv'  # Use product name as part of the file name
df.to_csv(csv_filename, index=False)

print(f"Reviews saved to {csv_filename}")
# Close the driver
driver.quit()




Product ID (Name): OPPO F27 Pro+ (Dusk Pink, 256 GB)  (8 GB RAM)
No more pages found or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76849B645+29573]
	(No symbol) [0x00007FF768410470]
	(No symbol) [0x00007FF7682CB6EA]
	(No symbol) [0x00007FF76831F815]
	(No symbol) [0x00007FF76831FA6C]
	(No symbol) [0x00007FF76836B917]
	(No symbol) [0x00007FF76834733F]
	(No symbol) [0x00007FF7683686BC]
	(No symbol) [0x00007FF7683470A3]
	(No symbol) [0x00007FF7683112DF]
	(No symbol) [0x00007FF768312441]
	GetHandleVerifier [0x00007FF7687CC58D+3375821]
	GetHandleVerifier [0x00007FF768817987+3684039]
	GetHandleVerifier [0x00007FF76880CDAB+3640043]
	GetHandleVerifier [0x00007FF76855B7C6+816390]
	(No symbol) [0x00007FF76841B77F]
	(No symbol) [0x00007FF7684175A4]
	(No symbol) [0x00007FF768417740]
	(No symbol) [0x00007FF76840659F]
	BaseThreadInitThunk [0x00007FFBB636257D+29]
	RtlUserThreadStart [0x00007FFBB6D2AF28+40]

Product ID: OPPO F27 Pro+ (Dusk Pink, 256 GB)  (8 GB RAM)
Rating: 5


In [35]:
import pandas as pd

# List of your CSV file paths
csv_files = [
    'C:/Users/Siva/Motorola Edge 50 Pro 5G with 125W Charger (Luxe Lavender, 256 GB)  (12 GB RAM)_reviews.csv',
    'C:/Users/Siva/SAMSUNG Galaxy S23 FE (Mint, 128 GB)  (8 GB RAM)_reviews.csv',
    'C:/Users/Siva/Google Pixel 7a (Snow, 128 GB)  (8 GB RAM)_reviews.csv',
    'C:/Users/Siva/OPPO F27 Pro+ (Dusk Pink, 256 GB)  (8 GB RAM)_reviews.csv',
    'C:/Users/Siva/Nothing Phone (2a) Plus (Black, 256 GB)  (12 GB RAM)_reviews.csv'
]

# Initialize an empty list to hold DataFrames
dataframes = []

# Read each CSV file and append the DataFrame to the list
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
        print(f"{file} loaded successfully.")
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('final_project_data.csv', index=False)

print("All files have been concatenated and saved as 'combined_data.csv'.")


C:/Users/Siva/Motorola Edge 50 Pro 5G with 125W Charger (Luxe Lavender, 256 GB)  (12 GB RAM)_reviews.csv loaded successfully.
C:/Users/Siva/SAMSUNG Galaxy S23 FE (Mint, 128 GB)  (8 GB RAM)_reviews.csv loaded successfully.
C:/Users/Siva/Google Pixel 7a (Snow, 128 GB)  (8 GB RAM)_reviews.csv loaded successfully.
C:/Users/Siva/OPPO F27 Pro+ (Dusk Pink, 256 GB)  (8 GB RAM)_reviews.csv loaded successfully.
C:/Users/Siva/Nothing Phone (2a) Plus (Black, 256 GB)  (12 GB RAM)_reviews.csv loaded successfully.
All files have been concatenated and saved as 'combined_data.csv'.


In [39]:
import pandas as pd

df = pd.read_csv('D:/final project/final_project_data.csv')

df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_id   442 non-null    object 
 1   review_text  442 non-null    object 
 2   rating       412 non-null    float64
dtypes: float64(1), object(2)
memory usage: 10.5+ KB


,product_id,review_text,rating
0,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,Do not believe the negative reviews. This is a...,5.0
1,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,Reviewing after 2days of using Moto Edge 50pro...,4.0
2,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,Very nice camera,5.0
3,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,I have been using this mobile for the last 6 d...,5.0
4,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,"Super camera\nLove you moto....,.................",5.0


In [41]:
missing_values = df.isnull().sum()
print(missing_values)


product_id      0
review_text     0
rating         30
dtype: int64


In [45]:
import re

# Step 1: Handle missing ratings by filling them with the average rating
df['rating'].fillna(df['rating'].mean(), inplace=True)

# Step 2: Clean the review text by removing special characters, extra spaces, and newlines
def clean_text(text):
    text = text.replace('\n', ' ')  # Replace newlines with spaces
    text = re.sub(r'[,.\'"]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip().lower()  # Convert to lowercase and remove leading/trailing spaces

df['cleaned_review_text'] = df['review_text'].apply(clean_text)

# Display cleaned text and filled ratings
df_cleaned_head = df[['product_id', 'cleaned_review_text', 'rating']].head()
df_cleaned_head


,product_id,cleaned_review_text,rating
0,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,do not believe the negative reviews this is an...,5.0
1,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,reviewing after 2days of using moto edge 50pro...,4.0
2,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,very nice camera,5.0
3,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,i have been using this mobile for the last 6 d...,5.0
4,Motorola Edge 50 Pro 5G with 125W Charger (Lux...,super camera love you moto,5.0


In [61]:
import re

# Step 1: Fix encoding issues safely
def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8', errors='ignore')  # 'ignore' to bypass unfixable characters
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text

# Step 2: Remove emojis and special symbols
def remove_non_ascii(text):
    # Regular expression to remove all non-ASCII characters, including emojis
    return re.sub(r'[^\x00-\x7F]+', '', text)

# Step 3: Combine both steps into one function
def clean_text(text):
    text = fix_encoding(text)  # Fix encoding first
    text = remove_non_ascii(text)  # Remove emojis and non-ASCII characters
    return text

# Apply cleaning function to the review_text column
df['cleaned_review_text'] = df['review_text'].apply(clean_text)

# Display cleaned data
df[['review_text', 'cleaned_review_text']].head()


,review_text,cleaned_review_text
0,Do not believe the negative reviews. This is a...,Do not believe the negative reviews. This is a...
1,Reviewing after 2days of using Moto Edge 50pro...,Reviewing after 2days of using Moto Edge 50pro...
2,Very nice camera,Very nice camera
3,I have been using this mobile for the last 6 d...,I have been using this mobile for the last 6 d...
4,"Super camera\nLove you moto....,.................","Super camera\nLove you moto....,................."


In [65]:
df = df.drop(['review_text'], axis=1)

In [67]:
df = df.rename(columns={'cleaned_review_text': 'review_text'})

In [73]:
df.to_csv('D:/final project/cleaned_final_project_data.csv', index=False)